In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from nltk.corpus import stopwords
from collections import Counter
import string
import re
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer
from transformers import BertModel
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification

In [2]:
! pip install -U accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 53.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
df = pd.read_csv('clean_df.csv')
df.head(20)

,Unnamed: 0,Text,Language,Label
0,430988,i wouldnt dream of it just dont tell me stop,en,positive
1,492831,question for sec fans if ohio state pl...,en,positive
2,76161,would be a dream come true to join,en,positive
3,744328,you know whats even easier admitting y...,en,positive
4,673077,people asking where is sapnap dude he s figh...,en,positive
5,554887,thanks for sending this our way you can hel...,en,positive
6,464265,electricity is the worlds greatest innovation ...,en,positive
7,867889,so good i had to share check out all the item...,en,positive
8,389780,the fear the entire republican party has of t...,en,positive
9,116268,i had a dream that featured last night she w...,en,positive


In [3]:
df.groupby('Label').size()

Label
negative       198586
positive       198586
uncertainty    198586
dtype: int64

In [4]:
from torch.utils.data import Dataset

In [5]:
class CustomDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_len=512):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    label = torch.tensor(self.labels[idx])

    encoding = self.tokenizer(text, truncation=True, padding="max_length",
                              max_length=self.max_len)

    return {
        'input_ids': encoding['input_ids'],
        'attention_mask': encoding['attention_mask'],
        'labels': label
    }

In [6]:
!pip install transformers

In [7]:


device = "cuda"

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3).to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
X = df['Text'].tolist()

label2id = {'uncertainty': 2 ,'positive': 1, 'negative': 0}
id2label = {2: 'uncertainty' , 1: 'positive', 0: 'negative'}

y = df['Label'].map(label2id).tolist()

dataset = CustomDataset(X, y, tokenizer)

In [9]:
dataset[0].keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [10]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [11]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(example):
  labels = example.label_ids
  preds = example.predictions.argmax(-1)

  f1 = f1_score(labels, preds, average="weighted")
  acc = accuracy_score(labels, preds)

  return {'accuracy': acc, "f1": f1}

In [12]:
from transformers import Trainer, TrainingArguments
batch_size = 16
model_name = "distilbert_finetuned_setiment"

args = TrainingArguments(
    output_dir = "output",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size = batch_size,
    learning_rate = 2e-5,
    num_train_epochs = 1,
    evaluation_strategy = 'epoch'
)

In [13]:
trainer = Trainer(model=model,
                  args=args,
                  train_dataset = train_dataset,
                  eval_dataset = test_dataset,
                  compute_metrics=compute_metrics,
                  tokenizer = tokenizer)

In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.063400,0.057275,0.976442,0.976458


TrainOutput(global_step=29788, training_loss=0.06781632304431705, metrics={'train_runtime': 8834.7793, 'train_samples_per_second': 53.947, 'train_steps_per_second': 3.372, 'total_flos': 6.313588292260454e+16, 'train_loss': 0.06781632304431705, 'epoch': 1.0})

In [15]:
trainer.save_model(model_name)

In [17]:
# sentiment analysis with the pipeline
from transformers import pipeline


In [18]:
text = "i love this product"
pipe = pipeline('text-classification', model_name)
pipe(text)

[{'label': 'LABEL_1', 'score': 0.7321298122406006}]

In [19]:
tok = AutoTokenizer.from_pretrained(model_name)
mod = AutoModelForSequenceClassification.from_pretrained(model_name)

In [20]:
def get_prediction(text):
  input_ids = tok.encode(text, return_tensors='pt')
  output = mod(input_ids)

  preds = torch.nn.functional.softmax(output.logits, dim=-1)

  prob = torch.max(preds).item()

  idx = torch.argmax(preds).item()
  sentiment = id2label[idx]

  return {'sentiment':sentiment, 'prob':prob}

In [21]:
text = "i love this product"
get_prediction(text)

{'sentiment': 'positive', 'prob': 0.7321298122406006}

In [22]:
text = "i hate this product"
get_prediction(text)

{'sentiment': 'negative', 'prob': 0.8606947064399719}

In [23]:
model.save_pretrained("model")